## 3. Which airlines cover the most cities?

In [122]:
import plotly.graph_objects as go
import pandas as pd

path = "../data/"
df = pd.read_csv(path+"processed/flights_vs_cities.csv")
df.head()

,Airline,Airport,Latitude,Longitude
0,American Airlines,Phoenix Sky Harbor International Airport,33.434299,-112.012001
1,American Airlines,Dallas Fort Worth International Airport,32.896801,-97.038002
2,American Airlines,Los Angeles International Airport,33.942501,-118.407997
3,American Airlines,Dallas Fort Worth International Airport,32.896801,-97.038002
4,American Airlines,Los Angeles International Airport,33.942501,-118.407997


In [123]:
df["Count"] = 1
df = df.groupby(["Airline", "Airport", "Latitude", "Longitude"]).count().reset_index()
df.head()

,Airline,Airport,Latitude,Longitude,Count
0,Alaska Airlines,Adak Airport,51.877998,-176.645996,18
1,Alaska Airlines,Albuquerque International Sunport,35.040199,-106.609001,62
2,Alaska Airlines,Austin Bergstrom International Airport,30.194500,-97.669899,266
3,Alaska Airlines,Baltimore/Washington International Thurgood Ma...,39.175400,-76.668297,227
4,Alaska Airlines,Bellingham International Airport,48.792801,-122.538002,128


In [124]:
df["Text"] = df["Airport"] + "   (" + df["Count"].astype(str) + ")"

### 1: Multi sub plot maps

In [125]:
data = []
layout = dict(
    title = 'Airline Operations by City',
    autosize = False,
    width = 1200,
    height = 800,
    hovermode = "closest",
    legend = dict(
        x=0.7,
        y=-0.1,
        bgcolor="rgba(255, 255, 255, 0)",
        font = dict( size=11 ),
    )
)
airlines = df['Airline'].unique()

for i in range(len(airlines)):
    geo_key = 'geo'+str(i+1) if i != 0 else 'geo'
    lons = list(df[ df['Airline'] == airlines[i] ]['Longitude'])
    lats = list(df[ df['Airline'] == airlines[i] ]['Latitude'])
    # Walmart store data
    data.append(
        dict(
            type = 'scattergeo',
            showlegend=False,
            lon = lons,
            lat = lats,
            geo = geo_key,
            name = airlines[i],
           marker=dict(
            color=df[df["Airline"] == airlines[i]]["Count"],
            colorscale="Viridis",
            opacity=0.8,
            showscale=(i == 0),
            colorbar=dict(title="Count") if i == 0 else None
        ),
        hoverinfo="text",
        text=df[df["Airline"] == airlines[i]]["Text"]
        )
    )
    # Year markers
    data.append(
        dict(
            type = 'scattergeo',
            showlegend = False,
            lon = [-78],
            lat = [47],
            geo = geo_key,
            text = [airlines[i]],
            mode = 'text',
        )
    )
    layout[geo_key] = dict(
        scope = 'usa',
        showland = True,
        landcolor = 'rgb(229, 229, 229)',
        showcountries = False,
        domain = dict( x = [], y = [] ),
        subunitcolor = "rgb(255, 255, 255)",
    )


def draw_sparkline( domain, lataxis, lonaxis ):
    ''' Returns a sparkline layout object for geo coordinates  '''
    return dict(
        showland = False,
        showframe = False,
        showcountries = False,
        showcoastlines = False,
        domain = domain,
        lataxis = lataxis,
        lonaxis = lonaxis,
        bgcolor = 'rgba(255,200,200,0.0)'
    )

z = 0
COLS = 4
ROWS = 4
for y in reversed(range(ROWS)):
    for x in range(COLS):
        geo_key = 'geo'+str(z+1) if z != 0 else 'geo'
        layout[geo_key]['domain']['x'] = [float(x)/float(COLS), float(x+1)/float(COLS)]
        layout[geo_key]['domain']['y'] = [float(y)/float(ROWS), float(y+1)/float(ROWS)]
        z=z+1
        if z > 15:
            break

fig = go.Figure(data=data, layout=layout)

### Plot or Export

In [126]:
path = "../plots/"

# fig.show()
fig.write_html(path+"flights_vs_cities.html", auto_open=True)